<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>Challenge Loan Approval Prediction in Pennsylvania</h1>
    <h3>Entrenamiento y evaluación de los modelos</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar librerias](#1.-Importar-librerias)
2. [Variables globales y funciones auxiliares](#2.-Variables-globales-y-funciones-auxiliares)
3. [Carga del dataframe](#3.-Carga-del-dataframe)
4. [Selección de las características](#4.-Seleccion-de-las-caracteristicas)
5. [Carga de los datos y división en entrenamiento y test](#5.-Carga-de-los-datos-y-division-en-entrenamiento-y-test)
6. [Modelos seleccionados](#6.-Modelos-seleccionados)
    * 6.1 [Introducción](#6.1-Introduccion)
    * 6.2 [Clasificador Random Forest](#6.2-Clasificador-Random-Forest)
    * 6.3 [Clasificador MLP](#6.3-Clasificador-MLP)
7. [Exportar CSV](#7.-Exportar-CSV)
    * 7.1 [Clasificador Random Forest](#7.1-Clasificador-Random-Forest)
    * 7.2 [Clasificador MLP](#7.2-Clasificador-MLP)

# 1. Importar librerias

In [1]:
# General import and load data
import pandas as pd
import numpy as np

# Resampling
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Splitting
from sklearn.model_selection import train_test_split

# Estimators
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
# Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import make_scorer, f1_score, roc_curve, roc_auc_score

# Optimization
from sklearn.model_selection import GridSearchCV, PredefinedSplit

# Visualization
import matplotlib.pyplot as plt

# Time optimization
import time

print("Todas las librerias fueron correctamente importadas.")

Todas las librerias fueron correctamente importadas.


# 2. Variables globales y funciones auxiliares

Se fija un seed para todo el documento para fijar la aleatoriedad y así obtener resultados replicables.

In [2]:
seed = 42

Función utilizado para evaluar los distintos umbrales en la curva de ROC.

In [3]:
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Recall:', 1 - fpr[thresholds > threshold][-1])

Función utilizada para estandarizar los datos en función del caso de preprocesado escogido.

In [4]:
def standard_data(case, X):
    
    if case in {1, 3, 5}:
        prep = StandardScaler().fit(X)
        X_scaled = prep.transform(X)
        
    elif case in {2, 4, 6}:
        prep = ColumnTransformer([
            ('numericas', StandardScaler(), numeric_vars)
        ], remainder='passthrough').fit(X) 
        X_scaled = prep.transform(X) 
        
    else:
        raise ValueError("El valor de 'case' no es válido. Debe estar entre 1 y 6.")
    
    return X_scaled, prep

Función utilizada para muestrear los datos en función del caso de preprocesado escogido.

In [5]:
def resample_data(case, X, y, strategy_under, strategy_over, neighbors=5):
    
    if case in {1, 2}:
        ovsamp = SMOTE(sampling_strategy=strategy_over, k_neighbors=neighbors, random_state=seed)
        X_resampled, y_resampled = ovsamp.fit_resample(X, y)
        
    elif case in {3, 4}:
        unsamp = RandomUnderSampler(sampling_strategy=strategy_under, random_state=seed)
        X_resampled, y_resampled = unsamp.fit_resample(X, y)
        
    elif case in {5, 6}:
        unsamp = RandomUnderSampler(sampling_strategy=strategy_under, random_state=seed)
        X_undersampled, y_undersampled = unsamp.fit_resample(X, y)
        ovsamp = SMOTE(sampling_strategy=strategy_over, k_neighbors=neighbors, random_state=seed)
        X_resampled, y_resampled = ovsamp.fit_resample(X_undersampled, y_undersampled)
        
    else:
        raise ValueError("El valor de 'case' no es válido. Debe estar entre 1 y 6.")
    
    return X_resampled, y_resampled

# 3. Carga del dataframe

Se cargan los datos de la ruta *formated/train_exportado.csv*, los cuales son los datos ya procesados por uno de nuestros compañeros.

In [6]:
url = "formated/train_exportado.csv"
df = pd.read_csv(url)

print("Datos cargados correctamente\n")

Datos cargados correctamente



También cargamos el test que debemos predecir para Kaggle de la ruta *formated/test_exportado.csv*.

In [7]:
url = "formated/test_exportado.csv"
df_test = pd.read_csv(url)

print("Test cargado correctamente\n")

Test cargado correctamente



# 4. Seleccion de las caracteristicas

Se seleccionan todas las características a excepción de las categoricas sin codificadar, al ya tener la misma információn codificada.

In [8]:
selected_features = [
    # Caracteristicas numericas
    'ApprovalFY', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross',

    # Caracteristicas categoricas binarias
    'NewExist_Binary', 'Franchise_Binary', 'UrbanRural_Binary',
    'RevLineCr_Binary', 'LowDoc_Binary', 'CreateJob_Binary',
    'RetainedJob_Binary',

    # Caracteristicas categoricas codificadas
    'Bank_Categorized_cod', 'BankState_Categorized_cod',
    'ApprovalFY_Grouped_cod', 'NoEmp_Grouped_cod', 
    'DisbursementGross_Grouped_cod',

     # Time-related features
    'ApprovalDate_quarter', 'DisbursementDate_quarter',

    # Caracteristicas relacionadas con BankState_Categorized tras un One Hot Encoding
    'BankState_DE', 'BankState_IL', 'BankState_OH', 
    'BankState_Otros', 'BankState_RI'
]

De las cuales, numéricas son las siguientes:

In [9]:
numeric_vars = ['ApprovalFY', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross']

# 5. Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *Accept* en la variable y.

In [10]:
X = df[selected_features] 
y = df['Accept'].values

Se separa ya en entrenamiento y test para no falsear los datos del test al realizar el balanceo.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

A la hora de subir el modelo a Kaggle, ya se nos da dividido el entrenamiento y el test. Por ello, se utilizarán todos los datos tratados como entrenamiento (aunque también se le aplicará el mejor caso de preprocesado para cada modelo).

In [12]:
X_train_final = X
y_train_final = y

X_test_final = df_test[selected_features] 

# 6. Modelos seleccionados

## 6.1 Introduccion

En el anterior notebook se evaluaron 10 modelos tanto por F1 score como por ROC-AUC score. Los resultados de mayor a menor de los modelos respecto a la F1-Score fueron los siguientes:
* Random Forest: 0.867449
* MLP: 0.811513
* DecisionTree: 0.790538
* KNN: 0.767966
* LinearSVC: 0.664541
* LogReg: 0.663316
* SVC_sigmoid: 0.617281
* GaussianNB: 0.568654
* SVC_poly: 0.526099
* SVC_rbf: 0.411304

En cambio, los resultados de mayor a menor de los modelos respecto a la ROC-AUC Score fueron los siguientes:
* Random Forest: 0.934825
* MLP: 0.884770
* KNN: 0.874725
* DecisionTree: 0.795805
* LinearSVC: 0.715244
* LogReg: 0.715162
* GaussianNB: 0.705878
* SVC_rbf: 0.692410
* SVC_poly: 0.658396
* SVC_sigmoid: 0.556961

Se decidió entrenar y optimizar los hiperparámetros de cuatro modelos. Los modelos escogidos fueron Random Forest, MLP, KNN y LinearSVC, es decir, los cuatro primeros tanto por F1-score como por ROC-AUC score (se ha excluido Decision Tree por ya estar probando un clasificador basado en árbol de decisión, y en un principio mejor).

## 6.2 Clasificador LinearSVC

### 6.2.1 Seleccion del caso de preprocesado

Seleccion del caso de preprocesado.

In [13]:
# case - caso de preprocesado seleccionado, valores posibles: 1, 2, 3, 4, 5, 6
case = 3

Hiperparametros base para cada caso.

In [14]:
if case in {1, 2}:
    strategy_under = 0
    strategy_over = 1
    neighbors = 5
    
elif case in {3, 4}:
    strategy_under = 0.8
    strategy_over = 0
    neighbors = 0
    
elif case in {5, 6}:
    strategy_under = 0.25
    strategy_over = 0.5
    neighbors = 5

Se estandarizan los datos en función del caso.

In [15]:
X_scaled, prep = standard_data(case, X_train)

Se muestrean los datos en función del caso.

In [16]:
X_resampled, y_resampled = resample_data(case, X_scaled, y_train, strategy_under, strategy_over, neighbors)

Se reducen las dimensionalidades mediante un *PCA*.

In [17]:
pca = PCA()
X_pca_full = pca.fit_transform(X_resampled)

cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
n_comp = np.argmax(cumulative_variance_ratio >= 0.95) + 1
X_pca = X_pca_full[:, :n_comp]

Al igual que se ha realizado distintos preprocesados al entrenamiento, hay que realizarlos al conjunto de test para obtener resultados coherentes.

No es necesario realizar las tecnicas de muestreo ya que eso son técnicas para balancear nuestro conjunto de entrenamiento. Solamente es necesario realizar la misma estandarización de las características y aplicar la misma reducción de dimensionalidad.

In [18]:
X_test_scaled = prep.transform(X_test)
X_test_pca_full = pca.transform(X_test_scaled)
X_test_pca = X_test_pca_full[:, :n_comp]

### 6.2.2 Definicion

Definimos el modelo básico.

In [19]:
model_SVC = LinearSVC()

### 6.2.3 Entrenamiento y evaluacion

Entrenamos el modelo y comprobamos su exactitud.

In [20]:
model_SVC.fit(X_pca, y_resampled)

predicted_SVC = model_SVC.predict(X_test_pca)
expected_SVC = y_test

In [21]:
accuracy_score(expected_SVC, predicted_SVC)

0.7204872646733111

### 6.2.4 Null accuracy

Ahora se va a evaluar si  el modelo siempre predice la clase más frecuente.

In [22]:
s_y_test = pd.Series(y_test)
s_y_test.value_counts()

1    3765
0     750
Name: count, dtype: int64

In [23]:
y_test.mean()

0.8338870431893688

In [24]:
1 - y_test.mean() 

0.1661129568106312

In [25]:
max(y_test.mean(), 1 - y_test.mean())

0.8338870431893688

In [26]:
s_y_test.value_counts().head(1) / len(y_test)

1    0.833887
Name: count, dtype: float64

Nuestra exactitud es de 7204, la cual es peor que la *null accuracy*, es decir, ahora mismo no tenemos un buen modelo ya que se predecirían un mayor número de resultados en el caso de siempre predecir la clase mayoritaria.

### 6.2.5 Matriz de confusion y F-score

Se muestra la matriz de confusión y el informe de clasificación.

In [27]:
print(confusion_matrix(expected_SVC, predicted_SVC))

[[ 410  340]
 [ 922 2843]]


In [28]:
print(classification_report(expected_SVC, predicted_SVC))

              precision    recall  f1-score   support

           0       0.31      0.55      0.39       750
           1       0.89      0.76      0.82      3765

    accuracy                           0.72      4515
   macro avg       0.60      0.65      0.61      4515
weighted avg       0.80      0.72      0.75      4515



Se observa lo que hemos estado comentando durante todo el documento, tenemos un f1-score del 0.82 para los 1s mientras que estamos obteniendo un 0.39 para los 0s

### 6.2.7 - Optimizacion

Antes de nada, observamos cuales son los parámetros de nuestro modelo.

In [29]:
model_SVC.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': 'auto',
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

Ahora seleccionamos los parametros a evaluar mediante un GridSearchCV. Como se evalua mediante un *cross validation*, siempre escogeremos los casos 3 o 4 de preprocesado para falsear menos los datos.

Estos son los parámetros:

C: Controla la regularización; valores altos reducen el margen de error pero pueden causar sobreajuste.

loss: Función de pérdida utilizada.

penalty: Tipo de regularización aplicada.

max_iter: Número máximo de iteraciones para la convergencia del algoritmo.

tol: Tolerancia para la optimización.

multi_class: Estrategia para clasificación multiclase.

### 6.2.8 Entramiento y evaluacion con hiperparametros optimizados

#### - Definicion

Definimos los mejores parámetros (no se define con *grid_search.best_params_* para probar varias opciones de forma sencilla).

Definimos el modelo desagregando el diccionario.

In [30]:
final_SVC = LinearSVC(
    C=0.01,  
    class_weight='balanced', 
    dual=True, 
    intercept_scaling=1, 
    loss='squared_hinge',  
    max_iter=100000,  
    penalty='l2',
    random_state=42,  
    tol=0.0000001  
)

#### - Entrenamiento y evaluacion

Entrenamos el modelo y comprobamos su exactitud.

In [31]:
final_SVC.fit(X_pca, y_resampled)

predicted_For_opt = final_SVC.predict(X_test_pca)
expected_For_opt = y_test

In [32]:
accuracy_score(expected_For_opt, predicted_For_opt)

0.6682170542635659

#### - Matriz de confusion y F-score

La matriz de confusión y el informe de clasificación también es muy similar.

In [33]:
print(confusion_matrix(expected_For_opt, predicted_For_opt))

[[ 484  266]
 [1232 2533]]


In [34]:
print(classification_report(expected_For_opt, predicted_For_opt))

              precision    recall  f1-score   support

           0       0.28      0.65      0.39       750
           1       0.90      0.67      0.77      3765

    accuracy                           0.67      4515
   macro avg       0.59      0.66      0.58      4515
weighted avg       0.80      0.67      0.71      4515



### 6.2.9 Conclusiones

A continuación se muestra los resultados de los modelos optimizados y sin optimizar. Cabe recalcar que se probaron otros cuatro conjuntos de hiperparámetros para la mayoria de los casos de preprocesado, pero se acabó considerando absurdo porque mejoraba o empeoraba 0.01 en diversos valores.

Caso 1 : 

              precision    recall  f1-score   support

           0       0.28      0.63      0.39       750
           1       0.90      0.67      0.77      3765

    accuracy                           0.67      4515

Caso 2 : 

              precision    recall  f1-score   support

           0       0.26      0.68      0.38       750
           1       0.91      0.62      0.73      3765

    accuracy                           0.63      4515

Caso 3 : 

              precision    recall  f1-score   support

           0       0.28      0.65      0.39       750
           1       0.90      0.67      0.77      3765

    accuracy                           0.67      4515

Caso 4 : 

              precision    recall  f1-score   support

           0       0.26      0.68      0.38       750
           1       0.91      0.62      0.73      3765

    accuracy                           0.63      4515

Caso 5 : 

              precision    recall  f1-score   support

           0       0.28      0.64      0.39       750
           1       0.90      0.67      0.77      3765

    accuracy                           0.67      4515

Caso 6 : 

              precision    recall  f1-score   support

           0       0.26      0.69      0.38       750
           1       0.91      0.62      0.74      3765

    accuracy                           0.63      4515   

La mayoria de los casos dan resultados similares

# 7. Exportar CSV

## 7.1 Clasificador SVC

### 7.1.1 Seleccion del caso de preprocesado

Seleccion del caso de preprocesado.

In [35]:
# case - caso de preprocesado seleccionado, valores posibles: 1, 2, 3, 4, 5, 6
case = 4

Hiperparametros base para cada caso.

In [36]:
if case in {1, 2}:
    strategy_under = 0
    strategy_over = 1
    neighbors = 5
    
elif case in {3, 4}:
    strategy_under = 0.9
    strategy_over = 0
    neighbors = 0
    
elif case in {5, 6}:
    strategy_under = 0.25
    strategy_over = 0.5
    neighbors = 5

Se estandarizan los datos en función del caso.

In [37]:
X_scaled_final, prep_final = standard_data(case, X_train_final)

Se muestrean los datos en función del caso.

In [38]:
X_resampled_final, y_resampled_final = resample_data(case, X_scaled_final, y_train_final, strategy_under, strategy_over, neighbors)

Se reducen las dimensionalidades mediante un *PCA*.

In [39]:
pca_final = PCA()
X_pca_full_final = pca_final.fit_transform(X_resampled_final)

cumulative_variance_ratio_final = np.cumsum(pca_final.explained_variance_ratio_)
n_comp_final = np.argmax(cumulative_variance_ratio_final >= 0.95) + 1
X_pca_final = X_pca_full_final[:, :n_comp_final]

Se preprocesa el conjunto de test.

In [40]:
X_test_scaled_final = prep_final.transform(X_test_final)
X_test_pca_full_final = pca_final.transform(X_test_scaled_final)
X_test_pca_final = X_test_pca_full_final[:, :n_comp_final]

### 7.1.2 Entrenamiento y exportacion de las predicciones

Se entrena con los nuevos valores de train y test, y se modifica el umbral de probabilidades. En un principio, el valor del umbral era el de la variable *op_thres_Random*, pero se decidió finalmente que el valor del umbral fueran valores numéricos para que fuera más sencillo hacer diversas pruebas en Kaggle.

In [41]:
final_SVC.fit(X_pca_final, y_resampled_final)

probs_For_opt = final_SVC.predict(X_test_pca_final) 

Se crea un *dataframe* con las predicciones obtenidas con su correspondiente *id*.

In [42]:
df_accept_KNN = pd.DataFrame(probs_For_opt, columns=['Accept'])

In [43]:
df_id = df_test['id'].reset_index(drop=True)

In [44]:
df_final_KNN = pd.concat([df_id, df_accept_KNN], axis=1)

Mostramos los resultados obtenidos para tener una idea de la proporción de 1s y 0s de Kaggle.

In [45]:
df_final_KNN['Accept'].value_counts()

Accept
0    1808
1    1476
Name: count, dtype: int64

Se transforma el *dataframe* a un archivo *CSV*.

In [46]:
df_final_KNN.to_csv('./svc.csv', index=False)